In [1]:
%matplotlib inline

import torch
from torch.nn import *

import torchvision
from torchvision.transforms import *

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
from torchsummary import summary

class YoloLayer(Module):
    def __init__(self, layer_params, pool=True):
        super().__init__()
        layers = []
        for in_channels, out_channels, kernel_size, stride, activation in layer_params:
            layers.append(
                YoloConv(in_channels=in_channels, out_channels=out_channels,
                         kernel_size=kernel_size, stride=stride, activation=activation)
            )
        
        self.in_layers = Sequential(*layers)
        if pool:
            self.pool = MaxPool2d(kernel_size=(2, 2), stride=2)
        else:
            self.pool = Identity()
        
    def forward(self, x):
        x = self.in_layers(x)
        x = self.pool(x)
        return x

class YoloConv(Module):
    def __init__(self, in_channels, out_channels, kernel_size=(3, 3), stride=1, activation='relu'):
        super().__init__()
        self.conv = Conv2d(kernel_size=kernel_size, in_channels=in_channels, out_channels=out_channels,
                          stride=stride, padding=kernel_size[0] // 2)
        if activation.lower() == 'relu':
            self.activation = LeakyReLU(negative_slope=0.1, inplace=True)
        else:
            self.activation = Identity
            
    def forward(self, x):
        x = self.conv(x)
        self.activation(x)  # inplace
        return x
    
class YoloFinalLayer(Module):
    def __init__(self, activation='relu'):
        super().__init__()
        self.classifier0 = Linear(in_features=50176, out_features=4096)
        if activation.lower() == 'relu':
            self.act = LeakyReLU(negative_slope=0.1, inplace=True)
        else:
            self.act = Identity()
        self.dropout = Dropout(0.5)
        self.classifier1 = Linear(in_features=4096, out_features=1470)
        
    def forward(self, x):
        x = x.view(-1, 50176)
        x = self.classifier0(x)
        self.act(x)
        x = self.dropout(x)
        x = self.classifier1(x)
        x = x.view(-1, 30, 7, 7)
        return x
    
class YoloFeatureExtractor(Module):
    def __init__(self, activation='relu'):
        super().__init__()
        layers = [
            YoloLayer([
                # in, out, kernel, stride, apoolct
                (3, 64, (7, 7), 2, activation),
            ]),
            YoloLayer([
                (64, 192, (3, 3), 1, activation),
            ]),
            YoloLayer([
                (192, 128, (1, 1), 1, activation),
                (128, 256, (3, 3), 1, activation),
                (256, 256, (1, 1), 1, activation),
                (256, 512, (3, 3), 1, activation),
            ]),
            YoloLayer([
                (512, 256, (1, 1), 1, activation),
                (256, 512, (3, 3), 1, activation),
                (512, 256, (1, 1), 1, activation),
                (256, 512, (3, 3), 1, activation),
                (512, 256, (1, 1), 1, activation),
                (256, 512, (3, 3), 1, activation),
                (512, 256, (1, 1), 1, activation),
                (256, 512, (3, 3), 1, activation),
                (512, 512, (1, 1), 1, activation),
                (512, 1024, (3, 3), 1, activation),
            ]),
            YoloLayer([
                (1024, 512, (1, 1), 1, activation),
                (512, 1024, (3, 3), 1, activation),
                (1024, 512, (1, 1), 1, activation),
                (512, 1024, (3, 3), 1, activation),
            ], pool=False)
        ]
        
        self.layers = Sequential(*layers)
        
    def forward(self, x):
        x = self.layers(x)
        return x
            
class YoloClassifier(Module):
    def __init__(self, activation='relu'):
        super().__init__()
        layers = [
            YoloLayer([
                (1024, 1024, (3, 3), 1, 'relu'),
                (1024, 1024, (3, 3), 2, 'relu'),
            ], pool=False),
            YoloLayer([
                (1024, 1024, (3, 3), 1, 'relu'),
                (1024, 1024, (3, 3), 1, 'relu'),
            ], pool=False),
            YoloFinalLayer('relu')
        ]
        
        self.layers = Sequential(*layers)
        
    def forward(self, x):
        x = self.layers(x)
        return x
        

class YOLOv1(Module):
    def __init__(self):
        super().__init__()
        
        self.features = YoloFeatureExtractor('relu')
        self.classifiers = YoloClassifier('relu')
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifiers(x)
        return x
    
class YOLOv1Pretrainer(Module):
    def __init__(self):
        super().__init__()
        
        self.features = YoloFeatureExtractor('relu')
        self.classifiers = Sequential(
            AvgPool2d(kernel_size=(7, 7)),
            Flatten(),
            Linear(in_features=1024, out_features=1000)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.classifiers(x)
        return x

model = YOLOv1()
summary(model, input_size=(3, 448, 448), batch_size=2, device='cpu')
# model = YOLOv1Pretrainer()
# summary(model, input_size=(3, 224, 224), batch_size=2, device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [2, 64, 224, 224]           9,472
         LeakyReLU-2          [2, 64, 224, 224]               0
          YoloConv-3          [2, 64, 224, 224]               0
         MaxPool2d-4          [2, 64, 112, 112]               0
         YoloLayer-5          [2, 64, 112, 112]               0
            Conv2d-6         [2, 192, 112, 112]         110,784
         LeakyReLU-7         [2, 192, 112, 112]               0
          YoloConv-8         [2, 192, 112, 112]               0
         MaxPool2d-9           [2, 192, 56, 56]               0
        YoloLayer-10           [2, 192, 56, 56]               0
           Conv2d-11           [2, 128, 56, 56]          24,704
        LeakyReLU-12           [2, 128, 56, 56]               0
         YoloConv-13           [2, 128, 56, 56]               0
           Conv2d-14           [2, 256,

In [3]:
from glob import glob
from tqdm.auto import tqdm
from xml.etree import ElementTree

import torch
import os
import math

class VOCYOLOAnnotator(object):
    def __init__(self, annotation_root, image_root, model_cells=7, extension='jpg'):
        super().__init__()
        self.annotation_root = annotation_root
        self.image_root = image_root
        self.model_cells = model_cells
        
        self.annotation_files = glob(os.path.join(annotation_root, '*.xml'))
        self.labels = self.find_object_names()
        
    def find_object_names(self):
        object_map = {}
        for xml_filename in tqdm(self.annotation_files, desc='Annotation 내 Object Names 검색'):
            with open(xml_filename, 'r') as f:
                root = ElementTree.fromstring(f.read())
                for item in root.findall('object'):
                    object_name = item.find('name').text.strip()
                    object_map[object_name] = 1
        return list(sorted(object_map.keys()))
    
    def parse_annotation(self):
        annotations = []
        for xml_filename in tqdm(self.annotation_files,  desc='Annotation 검색'):
            with open(xml_filename, 'r') as f:
                root = ElementTree.fromstring(f.read())
                size = root.find('size')
                
                filename = root.find('filename').text.strip()
                filepath = os.path.join(self.image_root, filename)
                image_width, image_height = int(size.find('width').text), int(size.find('height').text)
                objects = []
                for item in root.findall('object'):
                    object_name = item.find('name').text.strip()
                    object_id = self.labels.index(object_name)
                    
                    object_bndbox = item.find('bndbox')
                    (xmin, ymin, xmax, ymax) = [
                        float(object_bndbox.find(key).text) for key in ['xmin', 'ymin', 'xmax', 'ymax']
                    ]
                    
                    assert(object_id != -1)
                    
                    xmin_norm, ymin_norm = xmin / image_width, ymin / image_height
                    width, height = xmax - xmin, ymax - ymin
                    width_norm, height_norm = width / image_width, height / image_height
                    xcenter_norm, ycenter_norm = xmin_norm + width_norm / 2, ymin_norm + height_norm / 2
                    
                    # dynamic range tricks
                    # changes dynamic range from 0.0 to 7.0 and do floor()
                    # -> results 0, 1, 2, 3, 4, 5, 6!
                    cell_idx_x = math.floor(xcenter_norm * self.model_cells)
                    cell_idx_y = math.floor(ycenter_norm * self.model_cells)
                    
                    cell_pos_x_norm = (xcenter_norm - (cell_idx_x / self.model_cells))
                    cell_pos_y_norm = (ycenter_norm - (cell_idx_y / self.model_cells))
                    
                    objects.append([object_id, cell_idx_x, cell_idx_y, cell_pos_x_norm, cell_pos_y_norm, width_norm, height_norm])
                
                annotations.append((filepath, objects))
            
        return annotations

In [4]:
annotator = VOCYOLOAnnotator(
    annotation_root=r'C:\Development\dataset\VOCdevkit\VOC2007\Annotations',
    image_root=r'C:\Development\dataset\VOCdevkit\VOC2007\JPEGImages'
)

annotations = annotator.parse_annotation()
print("Annotation[0]:", annotations[0][0])
print("Annotation[1]:", annotations[0][1])

Annotation 내 Object Names 검색:   0%|          | 0/5011 [00:00<?, ?it/s]

Annotation 검색:   0%|          | 0/5011 [00:00<?, ?it/s]

Annotation[0]: C:\Development\dataset\VOCdevkit\VOC2007\JPEGImages\000005.jpg
Annotation[1]: [[8, 4, 5, 0.01557142857142857, 0.01904761904761898, 0.122, 0.3413333333333333], [8, 2, 5, 0.13228571428571434, 0.13371428571428567, 0.176, 0.288], [8, 0, 5, 0.072, 0.10971428571428565, 0.124, 0.3466666666666667], [8, 3, 4, 0.10742857142857148, 0.08590476190476193, 0.108, 0.28], [8, 4, 3, 0.017571428571428682, 0.11276190476190479, 0.07, 0.09066666666666667]]


In [5]:
class VOCYolo(torch.utils.data.Dataset):
    def __init__(self, labels, annotations, boxes=2, grid_size=7, transform=None):
        super(VOCYolo, self).__init__()
        self.labels = labels
        self.annotations = annotations
        self.transform = transform
        self.boxes = boxes
        self.grid_size = grid_size
        self.classes = 20  # fixed as it's VOC dataset!
        self.one_hot = torch.eye(self.classes)
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        filepath, annotation = self.annotations[idx]
        image = Image.open(filepath).convert('RGB')  # case if image is grayscale
        label = torch.zeros((5 + self.classes, self.grid_size, self.grid_size), dtype=torch.float)
        
        # fill label
        for (class_id, cell_idx_x, cell_idx_y, cell_pos_x, cell_pos_y, width, height) in annotation:
            if label[4, cell_idx_y, cell_idx_x] != 1.0:
                label[:5, cell_idx_y, cell_idx_x] = torch.from_numpy(np.array([cell_pos_x, cell_pos_y, width, height, 1.0], dtype=np.double))
                label[5:, cell_idx_y, cell_idx_x] = self.one_hot[class_id]
        
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [6]:
class YoloLoss(Module):
    def __init__(self, lambda_coord, lambda_noobj):
        super().__init__()
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj
        self.eps = 1e-15 # for sqrt(0), we will do sqrt(0 + eps)
        
    def forward(self, input: Tensor, target: Tensor) -> Tensor:
        assert((input.shape[1] - 20) % 5 == 0)
        bboxes = (input.shape[1] - 20) // 5
        
        # applies outer-model sigmoid function
        input = torch.sigmoid(input)
        
        # target: [cell_pos_x, cell_pos_y, width, height, 1.0] * 7 * 7
        object_gt_exist_mask = target[:, 4:5, :, :] == 1
        responsible_bbox_index_mask = YoloLoss.get_responsible_bbox_predictor_mask(input, target, bboxes)
        
        loss = 0
        for bbox_id in range(bboxes):
            current_box = input[:, 5 * bbox_id:5 * (bbox_id + 1), :, :]
            
            # we should use with object_gt_exist_mask
            # because non-object-existant mask will have zero index
            current_box_responsible_mask = responsible_bbox_index_mask == bbox_id
            
            #! TODO: iterate over xy_loss and find out that object_gt_exist_mask works well
            xy_loss = torch.square(current_box[:, :2, :, :] - target[:, :2, :, :]) * object_gt_exist_mask * current_box_responsible_mask
            loss += self.lambda_coord * torch.sum(xy_loss)
            # print("xy_loss ", torch.sum(xy_loss))
            
            wh_loss = torch.square(torch.sqrt(current_box[:, 2:4, :, :] + self.eps) - torch.sqrt(target[:, 2:4, :, :] + self.eps)) * object_gt_exist_mask * current_box_responsible_mask
            loss += self.lambda_coord * torch.sum(wh_loss)
            # print("wh_loss: ", torch.sum(wh_loss))
            # print("wh_loss current_box sqrt", torch.sum(torch.sqrt(current_box[:, 2:4, :, :] + self.eps)))
            # print("wh_loss target sqrt", torch.sum(torch.sqrt(target[:, 2:4, :, :] + self.eps)))
            
            conf_obj_loss = torch.square(current_box[: 4:5, :, :] - target[:, 4:5, :, :]) * object_gt_exist_mask * current_box_responsible_mask
            conf_noobj_loss = torch.square(current_box[: 4:5, :, :] - target[:, 4:5, :, :]) * ~(object_gt_exist_mask * current_box_responsible_mask)
            loss += torch.sum(conf_obj_loss)
            loss += self.lambda_noobj * torch.sum(conf_noobj_loss)
            # print("conf_obj_loss: ", torch.sum(conf_obj_loss))
            # print("conf_noobj_loss: ", torch.sum(conf_noobj_loss))
            
        class_loss = torch.square(input[:, (5 * bboxes):, :, :] - target[:, 5:, :, :])
        loss += torch.sum(class_loss)
        # print("class_loss: ", torch.sum(class_loss))
            
        return loss
    
    @staticmethod
    def get_responsible_bbox_predictor_mask(input: Tensor, target: Tensor, bboxes: int) -> Tensor:
        ious = []
        for bbox_id in range(bboxes):
            current_box_xywh = input[:, 5 * bbox_id:5 * (bbox_id + 1) - 1, :, :]
            label_xywh = target[:, :4, :, :]
            
            # iou -> (B * 7 * 7)
            iou = YoloLoss.get_iou_xywh(current_box_xywh, label_xywh)
            ious.append(iou)
        
        # stacked_iou -> (B * 2 * 7 * 7)
        stacked_iou = torch.stack(ious, dim=1)
        # print(stacked_iou.shape, torch.argmax(stacked_iou, dim=1, keepdim=True).shape)
        
        return torch.argmax(stacked_iou, dim=1, keepdim=True)
    
    @staticmethod
    def get_iou_xywh(input_xywh: Tensor, label_xywh: Tensor) -> Tensor:
        # index_map -> [1, 2, 7, 7]
        index_map_x = torch.arange(0, 7).repeat(7)
        index_map_y = torch.repeat_interleave(torch.arange(0, 7), 7)
        index_map = torch.unsqueeze(torch.stack([index_map_y, index_map_x], dim=0).view(2, 7, 7), 0)
        
        if input_xywh.device.type == 'cuda':
            index_map = index_map.cuda(non_blocking=True)
        
        input_xy_global = (input_xywh[:, :2, :, :] + index_map) / 7
        input_width_half, input_height_half = (input_xywh[:, 2, :, :] / 2), (input_xywh[:, 3, :, :] / 2)
        input_xmin = input_xy_global[:, 0, :, :] - input_width_half  # x_center - width / 2
        input_xmax = input_xy_global[:, 0, :, :] + input_width_half
        input_ymin = input_xy_global[:, 1, :, :] - input_height_half
        input_ymax = input_xy_global[:, 1, :, :] + input_height_half
        
        label_xy_global = (label_xywh[:, :2, :, :] + index_map) / 7
        label_width_half, label_height_half = (label_xywh[:, 2, :, :] / 2), (label_xywh[:, 3, :, :] / 2)
        label_xmin = label_xy_global[:, 0, :, :] - label_width_half  # x_center - width / 2
        label_xmax = label_xy_global[:, 0, :, :] + label_width_half
        label_ymin = label_xy_global[:, 1, :, :] - label_height_half
        label_ymax = label_xy_global[:, 1, :, :] + label_height_half

        input_volume = input_xywh[:, 2, :, :] * input_xywh[:, 3, :, :]
        label_volume = label_xywh[:, 2, :, :] * label_xywh[:, 3, :, :]
        intersect_width = torch.minimum(input_xmax, label_xmax) - torch.maximum(input_xmin, label_xmin)
        intersect_height = torch.minimum(input_ymax, label_ymax) - torch.maximum(input_ymin, label_ymin)
        intersect_volume = intersect_width * intersect_height
        union_volume = input_volume + label_volume - intersect_volume
        
        return intersect_volume / union_volume

In [7]:
import torchvision.transforms
from torchvision.transforms import *

train_dataset = VOCYolo(
    annotator.labels,
    annotations,
    transform=transforms.Compose([
        transforms.Resize((448, 448)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.4547857, 0.4349471, 0.40525291],
            std=[0.12003352, 0.12323549, 0.1392444]
        )
    ])
)

train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=2, num_workers=0, pin_memory=True)

model = YOLOv1().float().cuda()

criterion = YoloLoss(lambda_coord=5, lambda_noobj=0.5)
scaler = torch.cuda.amp.GradScaler()
optimizer = torch.optim.Adam(model.parameters())
    
for i, (image, label) in enumerate(train_dataloader):
    with torch.cuda.amp.autocast():
        image = image.cuda(non_blocking=True)
        label = label.cuda(non_blocking=True)
        
    output = model(image)
    loss = criterion(output, label)
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad(set_to_none=True)
    
    print("Batch %d/%d: Loss %.6f" % (i, len(train_dataloader), loss))

Batch 0/2506: Loss 618.048279
Batch 1/2506: Loss 499.237732
Batch 2/2506: Loss 56.602154
Batch 3/2506: Loss 57.019966
Batch 4/2506: Loss 85.452438
Batch 5/2506: Loss 66.465179
Batch 6/2506: Loss 78.577988
Batch 7/2506: Loss 50.668087
Batch 8/2506: Loss 116.307594
Batch 9/2506: Loss 58.737648
Batch 10/2506: Loss 55.312134
Batch 11/2506: Loss 127.595505
Batch 12/2506: Loss 33.996422
Batch 13/2506: Loss 43.574802
Batch 14/2506: Loss 71.114807
Batch 15/2506: Loss 152.329086
Batch 16/2506: Loss 46.574753
Batch 17/2506: Loss 76.560638
Batch 18/2506: Loss 108.858917
Batch 19/2506: Loss 52.793404
Batch 20/2506: Loss 49.422722
Batch 21/2506: Loss 128.460236
Batch 22/2506: Loss 35.748432
Batch 23/2506: Loss 112.799667
Batch 24/2506: Loss 71.529457
Batch 25/2506: Loss 50.057350
Batch 26/2506: Loss 79.207367
Batch 27/2506: Loss 54.300194
Batch 28/2506: Loss 47.407223
Batch 29/2506: Loss 78.266174
Batch 30/2506: Loss 55.148899
Batch 31/2506: Loss 142.754776
Batch 32/2506: Loss 91.361008
Batch 33/25

Batch 266/2506: Loss 74.054733
Batch 267/2506: Loss 79.454926
Batch 268/2506: Loss 161.538147
Batch 269/2506: Loss 109.062759
Batch 270/2506: Loss 69.401093
Batch 271/2506: Loss 72.583176
Batch 272/2506: Loss 56.388828
Batch 273/2506: Loss 47.489166
Batch 274/2506: Loss 37.712456
Batch 275/2506: Loss 111.603836
Batch 276/2506: Loss 91.053268
Batch 277/2506: Loss 121.248108
Batch 278/2506: Loss 59.180359
Batch 279/2506: Loss 61.974438
Batch 280/2506: Loss 54.340183
Batch 281/2506: Loss 58.085903
Batch 282/2506: Loss 94.010544
Batch 283/2506: Loss 50.452454
Batch 284/2506: Loss 78.115417
Batch 285/2506: Loss 78.600258
Batch 286/2506: Loss 49.458721
Batch 287/2506: Loss 30.897366
Batch 288/2506: Loss 53.187279
Batch 289/2506: Loss 107.692429
Batch 290/2506: Loss 65.283562
Batch 291/2506: Loss 81.658882
Batch 292/2506: Loss 50.448631
Batch 293/2506: Loss 62.013943
Batch 294/2506: Loss 34.828739
Batch 295/2506: Loss 43.874695
Batch 296/2506: Loss 69.278831
Batch 297/2506: Loss 55.563477
Bat

Batch 529/2506: Loss 67.684402
Batch 530/2506: Loss 36.767952
Batch 531/2506: Loss 44.975563
Batch 532/2506: Loss 147.072876
Batch 533/2506: Loss 36.673431
Batch 534/2506: Loss 79.318756
Batch 535/2506: Loss 65.797989
Batch 536/2506: Loss 55.813705
Batch 537/2506: Loss 39.804695
Batch 538/2506: Loss 176.589752
Batch 539/2506: Loss 64.966515
Batch 540/2506: Loss 43.035301
Batch 541/2506: Loss 70.513016
Batch 542/2506: Loss 75.990189
Batch 543/2506: Loss 51.476345
Batch 544/2506: Loss 97.375435
Batch 545/2506: Loss 63.357605
Batch 546/2506: Loss 66.089828
Batch 547/2506: Loss 76.970200
Batch 548/2506: Loss 64.447952
Batch 549/2506: Loss 104.721893
Batch 550/2506: Loss 195.436050
Batch 551/2506: Loss 83.237564
Batch 552/2506: Loss 40.387856
Batch 553/2506: Loss 58.484230
Batch 554/2506: Loss 44.103485
Batch 555/2506: Loss 68.830643
Batch 556/2506: Loss 86.156403
Batch 557/2506: Loss 40.213058
Batch 558/2506: Loss 144.690186
Batch 559/2506: Loss 51.486324
Batch 560/2506: Loss 64.238403
Bat

Batch 793/2506: Loss 43.018063
Batch 794/2506: Loss 38.452492
Batch 795/2506: Loss 80.181580
Batch 796/2506: Loss 36.779701
Batch 797/2506: Loss 29.834206
Batch 798/2506: Loss 56.761375
Batch 799/2506: Loss 37.981735
Batch 800/2506: Loss 53.836891
Batch 801/2506: Loss 69.883469
Batch 802/2506: Loss 111.088150
Batch 803/2506: Loss 139.586090
Batch 804/2506: Loss 30.842525
Batch 805/2506: Loss 68.417542
Batch 806/2506: Loss 110.791069
Batch 807/2506: Loss 72.501961
Batch 808/2506: Loss 56.131218
Batch 809/2506: Loss 74.805351
Batch 810/2506: Loss 76.091736
Batch 811/2506: Loss 31.769049
Batch 812/2506: Loss 58.223461
Batch 813/2506: Loss 99.151031
Batch 814/2506: Loss 61.298836
Batch 815/2506: Loss 62.344406
Batch 816/2506: Loss 37.748604
Batch 817/2506: Loss 77.727890
Batch 818/2506: Loss 84.253975
Batch 819/2506: Loss 43.455719
Batch 820/2506: Loss 90.433502
Batch 821/2506: Loss 104.507858
Batch 822/2506: Loss 74.304123
Batch 823/2506: Loss 65.402000
Batch 824/2506: Loss 64.989487
Batc

Batch 1055/2506: Loss 42.324478
Batch 1056/2506: Loss 95.858109
Batch 1057/2506: Loss 88.060898
Batch 1058/2506: Loss 69.274948
Batch 1059/2506: Loss 140.874924
Batch 1060/2506: Loss 44.345860
Batch 1061/2506: Loss 63.483280
Batch 1062/2506: Loss 63.534939
Batch 1063/2506: Loss 128.690414
Batch 1064/2506: Loss 110.754410
Batch 1065/2506: Loss 39.196800
Batch 1066/2506: Loss 48.566360
Batch 1067/2506: Loss 105.348122
Batch 1068/2506: Loss 80.876251
Batch 1069/2506: Loss 100.856728
Batch 1070/2506: Loss 37.938110
Batch 1071/2506: Loss 51.584038
Batch 1072/2506: Loss 49.077271
Batch 1073/2506: Loss 103.273659
Batch 1074/2506: Loss 151.765701
Batch 1075/2506: Loss 99.773720
Batch 1076/2506: Loss 61.662212
Batch 1077/2506: Loss 55.350636
Batch 1078/2506: Loss 112.907883
Batch 1079/2506: Loss 87.386452
Batch 1080/2506: Loss 50.048431
Batch 1081/2506: Loss 33.008141
Batch 1082/2506: Loss 75.391754
Batch 1083/2506: Loss 41.007149
Batch 1084/2506: Loss 40.653427
Batch 1085/2506: Loss 65.886414


Batch 1311/2506: Loss 30.878880
Batch 1312/2506: Loss 77.042839
Batch 1313/2506: Loss 81.761749
Batch 1314/2506: Loss 98.839302
Batch 1315/2506: Loss 89.260582
Batch 1316/2506: Loss 111.548294
Batch 1317/2506: Loss 51.338398
Batch 1318/2506: Loss 90.543671
Batch 1319/2506: Loss 76.143318
Batch 1320/2506: Loss 44.906372
Batch 1321/2506: Loss 63.128101
Batch 1322/2506: Loss 192.604111
Batch 1323/2506: Loss 205.264694
Batch 1324/2506: Loss 45.434921
Batch 1325/2506: Loss 145.444000
Batch 1326/2506: Loss 202.293854
Batch 1327/2506: Loss 51.147419
Batch 1328/2506: Loss 113.965736
Batch 1329/2506: Loss 98.439476
Batch 1330/2506: Loss 65.788330
Batch 1331/2506: Loss 110.123451
Batch 1332/2506: Loss 96.730141
Batch 1333/2506: Loss 77.445145
Batch 1334/2506: Loss 50.380577
Batch 1335/2506: Loss 130.461243
Batch 1336/2506: Loss 47.734318
Batch 1337/2506: Loss 39.068459
Batch 1338/2506: Loss 185.366669
Batch 1339/2506: Loss 75.721718
Batch 1340/2506: Loss 61.327587
Batch 1341/2506: Loss 60.528404

Batch 1567/2506: Loss 97.401810
Batch 1568/2506: Loss 116.283440
Batch 1569/2506: Loss 35.287819
Batch 1570/2506: Loss 73.864845
Batch 1571/2506: Loss 49.474995
Batch 1572/2506: Loss 67.821098
Batch 1573/2506: Loss 50.035782
Batch 1574/2506: Loss 93.619453
Batch 1575/2506: Loss 93.406219
Batch 1576/2506: Loss 141.969757
Batch 1577/2506: Loss 45.722977
Batch 1578/2506: Loss 95.844437
Batch 1579/2506: Loss 58.391685
Batch 1580/2506: Loss 33.688980
Batch 1581/2506: Loss 78.935097
Batch 1582/2506: Loss 77.258163
Batch 1583/2506: Loss 57.500526
Batch 1584/2506: Loss 60.568314
Batch 1585/2506: Loss 33.124142
Batch 1586/2506: Loss 84.137718
Batch 1587/2506: Loss 60.237640
Batch 1588/2506: Loss 47.055500
Batch 1589/2506: Loss 51.871750
Batch 1590/2506: Loss 96.617470
Batch 1591/2506: Loss 69.691200
Batch 1592/2506: Loss 73.333824
Batch 1593/2506: Loss 50.217243
Batch 1594/2506: Loss 147.232895
Batch 1595/2506: Loss 95.361252
Batch 1596/2506: Loss 48.563824
Batch 1597/2506: Loss 51.425404
Batch

Batch 1823/2506: Loss 55.091042
Batch 1824/2506: Loss 61.435768
Batch 1825/2506: Loss 105.772324
Batch 1826/2506: Loss 40.023872
Batch 1827/2506: Loss 136.691086
Batch 1828/2506: Loss 54.049042
Batch 1829/2506: Loss 125.686539
Batch 1830/2506: Loss 140.376541
Batch 1831/2506: Loss 89.380081
Batch 1832/2506: Loss 38.238808
Batch 1833/2506: Loss 73.636269
Batch 1834/2506: Loss 120.680092
Batch 1835/2506: Loss 86.533661
Batch 1836/2506: Loss 38.117607
Batch 1837/2506: Loss 85.424324
Batch 1838/2506: Loss 162.822723
Batch 1839/2506: Loss 121.589973
Batch 1840/2506: Loss 159.474304
Batch 1841/2506: Loss 135.052521
Batch 1842/2506: Loss 111.034317
Batch 1843/2506: Loss 103.688904
Batch 1844/2506: Loss 58.844078
Batch 1845/2506: Loss 109.865852
Batch 1846/2506: Loss 83.301666
Batch 1847/2506: Loss 80.151749
Batch 1848/2506: Loss 109.911926
Batch 1849/2506: Loss 54.679665
Batch 1850/2506: Loss 79.761902
Batch 1851/2506: Loss 127.971863
Batch 1852/2506: Loss 109.134193
Batch 1853/2506: Loss 42.

Batch 2079/2506: Loss 111.495682
Batch 2080/2506: Loss 69.165451
Batch 2081/2506: Loss 53.962414
Batch 2082/2506: Loss 89.924637
Batch 2083/2506: Loss 77.201462
Batch 2084/2506: Loss 56.029152
Batch 2085/2506: Loss 126.141617
Batch 2086/2506: Loss 77.470879
Batch 2087/2506: Loss 51.621681
Batch 2088/2506: Loss 70.764687
Batch 2089/2506: Loss 101.282387
Batch 2090/2506: Loss 147.266785
Batch 2091/2506: Loss 50.393978
Batch 2092/2506: Loss 34.331596
Batch 2093/2506: Loss 76.892403
Batch 2094/2506: Loss 103.520958
Batch 2095/2506: Loss 38.303982
Batch 2096/2506: Loss 54.193504
Batch 2097/2506: Loss 72.840714
Batch 2098/2506: Loss 40.086628
Batch 2099/2506: Loss 122.906708
Batch 2100/2506: Loss 100.181335
Batch 2101/2506: Loss 124.931259
Batch 2102/2506: Loss 39.852417
Batch 2103/2506: Loss 151.879578
Batch 2104/2506: Loss 26.738544
Batch 2105/2506: Loss 69.543694
Batch 2106/2506: Loss 39.030861
Batch 2107/2506: Loss 70.509026
Batch 2108/2506: Loss 116.902191
Batch 2109/2506: Loss 70.03496

Batch 2335/2506: Loss 49.304031
Batch 2336/2506: Loss 67.583694
Batch 2337/2506: Loss 42.157417
Batch 2338/2506: Loss 35.328903
Batch 2339/2506: Loss 49.144886
Batch 2340/2506: Loss 97.293556
Batch 2341/2506: Loss 41.243385
Batch 2342/2506: Loss 27.932610
Batch 2343/2506: Loss 106.997833
Batch 2344/2506: Loss 45.324356
Batch 2345/2506: Loss 52.483662
Batch 2346/2506: Loss 99.057076
Batch 2347/2506: Loss 48.579624
Batch 2348/2506: Loss 68.477257
Batch 2349/2506: Loss 51.953674
Batch 2350/2506: Loss 34.600552
Batch 2351/2506: Loss 118.114349
Batch 2352/2506: Loss 149.179535
Batch 2353/2506: Loss 53.738556
Batch 2354/2506: Loss 77.055862
Batch 2355/2506: Loss 35.332268
Batch 2356/2506: Loss 78.516953
Batch 2357/2506: Loss 87.203110
Batch 2358/2506: Loss 49.967655
Batch 2359/2506: Loss 44.818932
Batch 2360/2506: Loss 138.761551
Batch 2361/2506: Loss 28.640545
Batch 2362/2506: Loss 190.725464
Batch 2363/2506: Loss 86.637680
Batch 2364/2506: Loss 37.897121
Batch 2365/2506: Loss 120.134399
Ba